# Исследование надежности заемщиков

**Описание проекта.**

Заказчик — кредитный отдел банка. Необходимо исследовать, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных:**

- `children` — количество детей в семье

- `days_employed` — общий трудовой стаж в днях

- `dob_years` — возраст клиента в годах

- `education` — уровень образования клиента

- `education_id` — идентификатор уровня образования

- `family_status` — семейное положение

- `family_status_id` — идентификатор семейного положения

- `gender` — пол клиента

- `income_type` — тип занятости

- `debt` — имел ли задолженность по возврату кредитов

- `total_income` — ежемесячный доход

- `purpose` — цель получения кредита

## 1. Откроем таблицу и изучим общую информацию о данных

**1. Импортируем библиотеку `pandas`. Прочитаем данные из csv-файла в датафрейм и сохраним в переменную `data`.**

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**2. Выведем первые семь строк.**

In [2]:
data.head(7)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем


**3. Выведем основную информацию о датасете.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**4.Выведем количество пропущенных значений для каждого столбца.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**5. Заполнение пропусков в столбце `total_income`**

Столбец `total_income` хранит данные о доходах и содержит 2174 пропущенных значений. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**6. Обработем аномальные значения в столбце `days_employed`.**

В данных могут встречаться артефакты (аномалии) - значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное значение трудового стажа в столбце `days_employed`. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`. 

In [6]:
data['days_employed'] = data['days_employed'].abs()

**7. Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для дальнейшего анализа. 

**8. Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**9. В столбце `children` есть два аномальных значения.Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**10. Еще раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

**11. Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**12. Убедимся, что все пропуски заполнены. Проверим себя еще раз и выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**12. Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**14. Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы.**

In [14]:
data['education'] = data['education'].str.lower()

**15. Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

## Категоризация данных.

**16.  На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

Например, кредитополучателю с доходом 25000 назначим категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E - до 30000 руб'
        elif 30001 <= income <= 50000:
            return 'D - от 30001 до 50000 руб'
        elif 50001 <= income <= 200000:
            return 'C - от 50001 до 200000 руб'
        elif 200001 <= income <= 1000000:
            return 'B - от 200001 до 1000000 руб'
        elif income >= 1000001:
            return 'A - выше 1000000'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**17. Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**18. Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.

Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим какие подстроги помогут  правильно определить категорию.

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследуем данные

**Исследуем, существует ли зависимость между количеством детей и возвратом кредита в срок?**

In [24]:
data_pivot_children = data.pivot_table(index = 'children', values = 'debt', aggfunc = ['count', 'sum', 'mean']) 
data_pivot_children.index.names = ['Количество детей']
data_pivot_children.columns = ['Количество выданных кредитов', 'Количество просроченных платежей', 'Доля просроченных платежей']
data_pivot_children = data_pivot_children.sort_values(by=['Доля просроченных платежей'], ascending=False)
display(data_pivot_children.style.format({'Доля просроченных платежей':'{:.2%}'}))


,Количество выданных кредитов,Количество просроченных платежей,Доля просроченных платежей
Количество детей,,,
4,41,4,9.76%
2,2052,194,9.45%
1,4808,444,9.23%
3,330,27,8.18%
0,14091,1063,7.54%
5,9,0,0.00%


**Вывод:** 
1) наибольшая вероятность по возвратам кредитов в срок у семей с пятью детьми, где доля просроченных платежей составляет 0.00%, однако следует учитывать фактор, что семьи с пятью детьми берут кредиты очень редко, такие семьи брали кредиты всего девять раз, что составляет менее 1% от всех выданных кредитов;

2) семьи без детей возвращают кредиты в срок чаще, чем семьи с детьми. Доля просроченных платежей для семей без детей составила 7.54%. Заметная зависимость между количеством детей и возвратом кредитов в срок отсутствует, доли просроченных платежей распределились почти равномерно между семьями с одим ребенком (9.23%), двумя (9.45%), и четырьмя детьми (9.76), при этом семьи с тремя детьми возвращают кредиты немного чаще (8.18%);



**Исследуем, существует ли зависимость между семейным положением и возвратом кредита в срок?**

In [25]:
data_pivot_status = data.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['count', 'sum', 'mean'])
data_pivot_status.columns = ['Количество выданных кредитов', 'Количество просроченных платежей', 'Доля просроченных платежей']
data_pivot_status.index.names = ['Семейный статус']
data_pivot_status = data_pivot_status.sort_values(by=['Доля просроченных платежей'], ascending=False)
display(data_pivot_status.style.format({'Доля просроченных платежей':'{:.2%}'}))

,Количество выданных кредитов,Количество просроченных платежей,Доля просроченных платежей
Семейный статус,,,
Не женат / не замужем,2796,273,9.76%
гражданский брак,4134,385,9.31%
женат / замужем,12261,927,7.56%
в разводе,1189,84,7.06%
вдовец / вдова,951,63,6.62%


**Вывод:** В результате проведенного исследования мы выяснили, что семейный статус влияет на вероятность возврата кредита в срок:
Легче всего возвращают кредиты в срок вдовцы, где доля просроченных платежей наименьшая и составляет 6.62%, а также граждане, состоящие в разводе (7.06%), и граждане, состоящие в официальном браке (7.56%), которые возвращают кредиты в срок немного реже, чем граждане, состоящие в разводе.
Самая высокая доля просроченных платежей у граждан, не состоящих в официальном браке (9.76%) и живущих гражданским браком (9.31%).

**Исследуем, существует ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [26]:
data_pivot_income = data.pivot_table(index = 'total_income_category', values = 'debt', aggfunc = ['count', 'sum', 'mean'])
data_pivot_income.columns = ['Количество выданных кредитов', 'Количество просроченных платежей', 'Доля просроченных платежей']
data_pivot_income.index.names = ['Категория по доходам']
data_pivot_income = data_pivot_income.sort_values(by=['Доля просроченных платежей'], ascending=False)
display(data_pivot_income.style.format({'Доля просроченных платежей':'{:.2%}'}))

,Количество выданных кредитов,Количество просроченных платежей,Доля просроченных платежей
Категория по доходам,,,
E - до 30000 руб,22,2,9.09%
C - от 50001 до 200000 руб,15921,1353,8.50%
A - выше 1000000,25,2,8.00%
B - от 200001 до 1000000 руб,5014,354,7.06%
D - от 30001 до 50000 руб,349,21,6.02%


**Вывод:**  В результате проведенного исследования мы выяснили, что группы граждан с доходом от 30001 до 50000 рублей (категория D) имеют самую высокую долю по возвратам кредитов в срок (6.02%). Граждане с высокими доходами от 200001 до 1000000 рублей(категория B)(доля возвратов 7.06%) и выше 1000000 рублей (категория А) возвращают кредиты немного реже (доля возвратов 8.00%), чем граждане из категории D.

Самая большая просрочка по кредитам у граждан из категории Е с доходом до 30 000 руб - доля просроченных платежей 9.09%, а также у граждан из категории C с доходом от 50001 до 200000 руб - доля просроченных платежей 8.50%.

Если сравнить две самые многочисленные выборки, это группа C и группа B, то можно сделать вывод, что граждане с доходом от 50001 до 200 000 рублей берут кредиты чаще всех: в три раза чаще, чем граждане из второй самой многочисленной выборки с доходом от 200001 до 1000000 руб, при этом граждане с меньшим доходом из группы С возвращают платежи по кредитам менее охотно, чем граждане с бОльшим доходом из группы B.

**Исследуем, как разные цели кредита влияют на его возврат в срок?**

In [27]:
data_pivot_purpose = data.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = ['count', 'sum', 'mean'])
data_pivot_purpose.columns = ['Количество выданных кредитов', 'Количество просроченных платежей', 'Доля просроченных платежей']
data_pivot_purpose.index.names = ['Цель кредита']
data_pivot_purpose = data_pivot_purpose.sort_values(by=['Доля просроченных платежей'], ascending=False)
display(data_pivot_purpose.style.format({'Доля просроченных платежей':'{:.2%}'}))

,Количество выданных кредитов,Количество просроченных платежей,Доля просроченных платежей
Цель кредита,,,
операции с автомобилем,4279,400,9.35%
получение образования,3988,369,9.25%
проведение свадьбы,2313,183,7.91%
операции с недвижимостью,10751,780,7.26%


**Вывод:** Наибольший процент возврата по кредитам в установленные сроки при таких целях, как операции с недвижимостью, где доля просроченных платежей составляет 7.26%, а также на проведение свадьбы - 7.91%.
Самая высокая доля просроченных платежей при операциях с автомобилями, составляет 9.35% и получение образования 9.25%.

Самыми проблемными выглядят автокредиты и кредиты на образование.

<div class="alert alert-info">
<b>Цель исследования:</b>
проверка гипотезы влияет ли семейное положение и количество детей клиента банка на факт погашения кредита в срок. 

<b>Входные данные от банка:</b> статистика о платёжеспособности клиентов.
    
На этапе предобработки данных мы заполнили пропуски медианнымы значениеми в столбцах с общим трудовым стажем и ежемесячным доходом, удалили все строки-дубликаты.

Также мы разделили кредитополучателей на категории, в зависимости от доходов:
0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001–1000000 — 'B';
1000001 и выше — 'A'.
    
Произвели оптимизацию столбца "цель кредита" (purpose_category), сгруппировав цели по четырем категориям:   
    
'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования'
    
<b>Общий вывод:</b>    
    
       
1. Самые многочисленные выборки, это семьи, которые чаще всего берут кредиты - семьи без детей (выдано 66% от всех выданных кредитов), семьи с одним (выдано 22.5% от всех выданных кредитов), и семьи с двумя детьми (выдано 9.6% от всех выданных кредитов).
    
  Многодетные семьи с тремя, четырьмя и пятью детьми берут кредиты реже, чем семьи без детей, или с одним, или с двумя детьми.

  Наиболее надежные и выгодные заемщики - это семьи без детей, где доля просроченных платежей составила 7.54%.
  Семьи с одним или двумя детьми менее надежные заемщики, где доля просроченных платежей выше и составляет 9.23 и 9.45%     соответственно.

  Семьи с тремя детьми возвращают кредиты немного чаще (8.18%), а берут кредиты неохотно.
    
  Наибольшая вероятность по возвратам кредитов в срок у семей с пятью детьми, где доля просроченных платежей наименьная   составляет 0.00%, однако следует учитывать фактор, что семьи с пятью детьми берут кредиты очень редко, такие семьи брали кредиты всего девять раз, что составляет менее 1% от всех выданных кредитов.
    
2. Состоящие или состоявшие в официальном браке заемщики лучше всего возвращают кредиты в срок.
    Для вдовцов доля просроченных платежей наименьшая и составляет 6.62%.
    Для граждан, состоявщих в разводе доля просроченных платежей немного выше и составляет 7.06%.
    Для граждан, состоявщих в официальном браке доля просроченных платежей почти такая же, как для граждан, состоящих в разводе, и этом, граждане состоящие в официальном браке берут кредиты наиболее часто, а именно в 57.47% случаев.
    

3. В результате проведенного исследования мы выяснили, самые надежные заемщики, это граждане с доходом от 30001 до 50000 рублей (категория D), где доля по возвратам кредитов в срок составила 6.02%, однако такие граждане редко берут кредиты (процент выданных кредитов составил всего 1.63 %).
    Чаще всего берут кредиты граждане из категории С с доходом от 50001 до 200000 рублей (процент выданных кредитов составил 74.63%), а также граждане из категории B с доходом от 200001 до 1000000 рублей (процент выданных кредитов составил 23.5%), при этом раждане с меньшим доходом из группы С возвращают платежи по кредитам менее охотно (доля просроченных платежей 8.5%), чем граждане с бОльшим доходом из группы B (доля просроченных платежей 7.06%)
 
 Самые ненадежные заемщики, это граждане из категории Е с доходом до 30 000 руб - доля просроченных платежей 9.09%, однако такие граждане предпочитают избегать кредитов (процент выданных кредитов составил 0.1%), как и граждане с наибольшим доходом свыше 1000000 руб из категории A (процент выданных кредитов составил 0.11%), где доля просроченных платежей составила 8%.
    
4. Чаще всего граждане берут кредиты на сделки с недвижимостью (процент выданных кредитов составил 50.4%), и при этом выданные кредиты наиболее надежные, так как имеют наибольший процент возврата по кредитам в установленные сроки, где доля просроченных платежей наименьшая и составляет 7.26%.
    Самые ненадежные заемщики граждане, которые берут кредиты на операции с автомобилями(процент выданных кредитов 20%), где доля просроченных платежей самая высокая и составляет 9.35%, а также на получение образования, где доля просроченных платежей составила 9.25% (а процент выданных кредитов 18.69%), т.е. самыми проблемными выглядят автокредиты и кредиты на образование.
    
    В результате исследования можно сделать общий вывод, что идеальные заемщики для банка, это граждане, желающие взять ипотечный кредит, вдовцы или состоящие в разводе, без детей с официальным доходом от 30001 до 50 000 рублей, а также от 200 000 до 1 000 000 рублей.
    

    
    
</div>    